In [1]:
from gramgen import GGTrainer

from transformers import AutoModelForMaskedLM, TrainingArguments, Trainer
import torch

model_name = 'bert-base-uncased'

model = AutoModelForMaskedLM.from_pretrained(model_name)

from utils.datasets_config import get_dataset

dset = get_dataset('gigaword', model_name)
print(dset)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Found cached dataset gigaword (/home/ruihan/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/ruihan/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6/cache-b980698e6916dd2c.arrow
Loading cached processed dataset at /home/ruihan/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6/cache-cdacb1db7b56e323.arrow
Loading cached processed dataset at /home/ruihan/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6/cache-c1a4948f8e480a64.arrow


Number of gigaword training examples: 3803957
Number of gigaword validation examples: 189651
Number of gigaword testing examples: 1951
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'input_texts', 'label_texts'],
        num_rows: 3803957
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'input_texts', 'label_texts'],
        num_rows: 189651
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'input_texts', 'label_texts'],
        num_rows: 1951
    })
})


In [2]:
training_args = TrainingArguments(
    # evaluation_strategy = "steps",
    output_dir= model_name + '-maskedlm',
    overwrite_output_dir=True,
    num_train_epochs=1,
    max_steps = 700_000,
    per_device_train_batch_size=4,
    save_steps=5_000,
    save_total_limit=20,
    prediction_loss_only=True,
    dataloader_num_workers=4,
    # learning_rate=3e-4,
    # logging_steps = 5,
    # eval_steps = 5,
    # metric_for_best_model = 'f1',
    # load_best_model_at_end=True,
)


In [3]:
def collate(batch):
    batch = ({k: torch.nn.utils.rnn.pad_sequence([dic[k] for dic in batch], batch_first=True, padding_value=1) for k in batch[0]})
    # batch['input_ids'][batch['input_ids'] == -100] = 1
    batch['attention_mask'] = (batch['input_ids'] != 1).long()
    batch['labels'] = torch.nn.functional.pad(batch['labels'], (0, batch['input_ids'].shape[1] - batch['labels'].shape[1], 0, 0), 'constant', 1)
    return batch

In [4]:
trainer = GGTrainer(
    model = model,
    args = training_args,
    train_dataset = dset['train'].shuffle(1234),
    eval_dataset = dset['validation'].shard(300, 1),
    data_collator = collate,
    # compute_metrics = lambda x: print(x),
    # callbacks = [EarlyStoppingCallback(early_stopping_patience=7)],
)


Loading cached shuffled indices for dataset at /home/ruihan/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6/cache-98e75d5f6ae6e6d1.arrow
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train(resume_from_checkpoint = False)

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: input_texts, label_texts. If input_texts, label_texts are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/mnt/rh/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3803957
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 700000
  Number of trainable parameters = 109514298


Step,Training Loss
500,5.907000
1000,5.192400
1500,5.169800
2000,5.117200
2500,5.165000
3000,5.530000
3500,5.599300
4000,5.513300
4500,5.217500
5000,5.088900


Saving model checkpoint to bert-base-uncased-maskedlm/checkpoint-5000
Configuration saved in bert-base-uncased-maskedlm/checkpoint-5000/config.json
Configuration saved in bert-base-uncased-maskedlm/checkpoint-5000/generation_config.json
Model weights saved in bert-base-uncased-maskedlm/checkpoint-5000/pytorch_model.bin
Saving model checkpoint to bert-base-uncased-maskedlm/checkpoint-10000
Configuration saved in bert-base-uncased-maskedlm/checkpoint-10000/config.json
Configuration saved in bert-base-uncased-maskedlm/checkpoint-10000/generation_config.json
Model weights saved in bert-base-uncased-maskedlm/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to bert-base-uncased-maskedlm/checkpoint-15000
Configuration saved in bert-base-uncased-maskedlm/checkpoint-15000/config.json
Configuration saved in bert-base-uncased-maskedlm/checkpoint-15000/generation_config.json
Model weights saved in bert-base-uncased-maskedlm/checkpoint-15000/pytorch_model.bin
Saving model checkpoint to be

In [ ]:
trainer.evaluate()